In [ ]:
from bs4 import BeautifulSoup

from selenium import webdriver 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager

from dataclasses import dataclass

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
driver.get("https://www.thrillist.com/philadelphia")

In [ ]:
html = driver.page_source
parser = BeautifulSoup(html, 'html.parser')

In [ ]:
cards = parser.find_all('div', class_='UniversalContentCardstyles__UCCTextBlock-sc-r8ldlw-0 hWJlON', recursive=True)[:4]

In [ ]:
@dataclass
class Article:
    activity: str
    href: str
    title: str
    description: str

In [ ]:
articles = []

In [ ]:
for card in cards:
    activityType = card.find_all('a', recursive=True)[0].text
    href = "https://www.thrillist.com" + card.find_all('a', recursive=True)[1]['href']
    h2 = card.find('h2', recursive=True).text
    h3 = card.find('h3', recursive=True).text
    
    newArticle = Article(activityType, href, h2, h3)
    articles.append(newArticle)

In [ ]:
articles

In [ ]:
driver.get(articles[0].href)

In [ ]:
html = driver.page_source
parser = BeautifulSoup(html, 'html.parser')

In [ ]:
content = parser.find('div', class_='NodeArticlestyles__ObscuredContentWrapper-sc-1dhoc8d-6 hUqleM', recursive=True)

In [ ]:
h2s = content.find_all('h2', recursive=True)

In [ ]:
len(h2s)

In [ ]:
@dataclass
class Suggestion:
    title: str
    href: str

In [ ]:
suggestions1 = []

BASE_URL = "https://www.thrillist.com"

In [ ]:
for h2 in h2s:
    title = h2.text
    a = h2.parent if h2.parent.name == 'a' else h2.find('a', recursive=True)
    if not a:
        continue
    href = a['href']
    href = href if 'http' in href else BASE_URL + href
    
    newSuggestion = Suggestion(title, href)
    suggestions1.append(newSuggestion)

In [ ]:
suggestions1

In [ ]:
import googlemaps

In [ ]:
client = googlemaps.Client("API_KEY_HERE")

In [ ]:
destination = client.find_place(
    f"{suggestions1[0].title} YOUR_CITY_HERE",
    "textquery",
    fields=["place_id"],
    location_bias="point:90,90",
    language="en-US",
)

destination

In [ ]:
destination_id = destination['candidates'][0]['place_id']

destination_id

In [ ]:
origin = client.find_place(
    "YOUR_HOME_ADDRESS_HERE",
    "textquery",
    fields=["business_status", "place_id"],
    location_bias="point:90,90",
    language="en-US",
)

origin

In [ ]:
origin_id = origin['candidates'][0]['place_id']

origin_id

In [ ]:
hours = client.place(
    destination_id,
    fields=["opening_hours"],
    language="en-US",
)

hours

In [ ]:
saturday = hours['result']['opening_hours']['weekday_text'][5]

saturday

In [ ]:
openText = ' '.join(saturday.split(' ')[1:])

openText

In [ ]:
walking = client.directions(
    f'place_id:{origin_id}',
    f'place_id:{destination_id}',
    mode="walking",
    units="metric",
    region="us",
)

walking

In [ ]:
walkingInfo = walking[0]['legs'][0]['duration']

walkingInfo

In [ ]:
transit = client.directions(
    f'place_id:{origin_id}',
    f'place_id:{destination_id}',
    mode="transit",
    units="metric",
    region="us",
)

transit

In [ ]:
transitInfo = transit[0]['legs'][0]['duration']

transitInfo

In [ ]:
shouldWalk = walkingInfo['value'] < transitInfo['value']

shouldWalk

In [ ]:
body = ""

emojisDict = {
    'eat': '🍔🍔🍔',
    'lifestyle': '🗿🗿🗿',
    'event': '🎪🎪🎪'
}

body += f"{emojisDict[articles[0].activity.lower()]} {articles[0].title}\n"

In [ ]:
for idx, suggestion in enumerate(suggestions1):
    body += '\n'
    body += f"{idx + 1}. "
    
    etaAndMode = ""
    if shouldWalk:
        etaAndMode = f"🚶‍♂️ {walkingInfo['text']}"
    else:
        etaAndMode = f"🚌 {transitInfo['text']}"
        
    body += f"{suggestion.title} ({openText}) - {etaAndMode} - {suggestion.href}"

In [ ]:
print(body)

In [ ]:
import os
from twilio.rest import Client


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = "YOUR_TWILIO_SID_HERE"
auth_token = "YOUR_TWILIO_TOKEN_HERE"
client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body=body,
                     from_='+YOUR_VIRTUAL_PHONE_NUMBER_HERE',
                     to='+RECEIVE_PHONE_NUMBER_HERE'
                 )

print(message.sid)

In [ ]:
print(message)

In [ ]:
print(message.status)